In [4]:
import sys 
sys.path.append("../")

import nerf_model
import dataloader

import torch 
import cv2
from PIL import Image
import itertools 
import numpy as np

In [ ]:
im_h = 5
im_w = 5
idxs = [(i,j) for i,j in itertools.product(np.arange(0,im_h), np.arange(0,im_w))]
idxs = torch.FloatTensor(idxs)
# print(idxs)
# print(idxs.shape)

In [ ]:
def photo_nerf_to_image(model, im_h, im_w): 
    idxs = [(i,j) for i,j in itertools.product(np.arange(0,im_h), np.arange(0,im_w))]
    idxs = torch.FloatTensor(idxs)
    N, _ = idxs.shape
    recon = []
    step = 4096
    for i in range(0, N, step):
        # break up whole tensor into sizeable chunks
        batch = idxs[i:i+step,:]
        rgb = model(batch)
        recon.append(rgb)
    recon = torch.cat(recon, axis=0).reshape((im_h, im_w, 3))
    return recon

model = nerf_model.ImageNeRFModel()
recon = photo_nerf_to_image(model, 400, 500)
print(recon.shape)

In [5]:
im_path = '../tests/test_data/grad_lounge.png'
bgr = cv2.imread(im_path, 1)
cv2_im = cv2.cvtColor(cv2.imread(im_path, 1), cv2.COLOR_BGR2RGB)
print(cv2_im.shape)

pil_im = np.array(Image.open(im_path))
print(pil_im.shape)

(403, 538, 3)
(403, 538, 3)


In [6]:
np.testing.assert_allclose(cv2_im, pil_im)

In [7]:
print(bgr[0,0,:])
print(pil_im[0,0,:])

[237 139  61]
[ 61 139 237]


In [8]:
print(pil_im.dtype)
print(bgr.dtype)

uint8
uint8


In [9]:
ds = dataloader.PhotoDataset(im_path)
ds[0]

(tensor([0., 0.]), tensor([ 1.9407,  0.3978, -0.7413]))